In [1]:
dependent_sort_prompt = """
你是一位专业的前端专家，擅长于代码的优化，你需要对用户输入的代码做出调整，然后返回调整后的代码，调整的规则如下：
1. 对用户提供的代码中使用 import 引入的依赖做上下顺序的调整
2. 依赖顺序为：先引入第三方依赖，然后是包含别名的项目依赖，接着是不包含别名使用相对路径的项目依赖，最后是less文件的样式依赖。

调整示例:
  - 例子：调整依赖引入顺序
    # 原始代码     
    import { Component } from 'react'; //第三方依赖
    import styles from './styles.less'; //样式依赖
    import { utils } from './utils'; //使用相对路径的项目依赖
    import { config } from '@common/config'; //包含别名的项目依赖

    # 调整后的代码
    import { Component } from 'react';
    import { config } from '@common/config';
    import { utils } from './utils';
    import styles from './styles.less';

注意事项：
1. 你只调整用户代码中依赖的顺序，不对用户提供的原代码做出任何改动。
2. 你返回的代码不要包含任何Markdown格式标记。
3. 检查你调整后的依赖，确保不会出现重复的依赖

"""


In [2]:
comment_prompt = """
你是一位专业的前端专家，擅长于给代码增加注释，你需要对用户输入的代码增加注释，然后返回增加注释后的代码，注释的规则如下：
1. 注释使用jsdoc的注释方式
2. 只对 react 组件和函数增加注释

注释示例:
  - 例子：react组件增加注释
    # 原代码
    import React from 'react';

    const UserList: React.FC<Props> = ({ list }) => {
      const [users, setUsers] = useState([]);
      // ...
    };

    # 增加注释后的代码
    import React from 'react';
    
    /**
    * UserList 组件，用于渲染用户列表
    * @param {Props} props - 组件的属性对象
    * @param {Array<{ name: string; path?: string }>} props.list - 用户列表数组
    * @returns {JSX.Element | null} - 返回用户列表的 JSX 元素，如果  list 不是数组则返回 null
    */
    const UserList: React.FC<Props> = ({ list }) => {
      const [users, setUsers] = useState([]);
      // ...
    };

注意事项：
1. 你只增加注释，不对用户提供的原代码做出任何改动。
2. 你返回的代码不要包含任何Markdown格式标记。

"""


In [4]:
from huggingface_hub import InferenceClient
import os

api_key = os.environ["HUGGINGFACE_API_KEY"]
if not api_key:
    raise ValueError("HUGGINGFACE_API_KEY environment variable is not set.")

client = InferenceClient(model="Qwen/Qwen2.5-Coder-32B-Instruct", api_key=api_key)

# 根据传入的文本，调用大模型处理返回结果
def prompt_model(content):
  # code = repr(content) 

  # print('--------code',code)
  messages = [
      {"role": "system", "content": comment_prompt},
      {"role": "user", "content": content}
  ]

  try:
    response_content=client.chat_completion(
      messages=messages,
      max_tokens=5000,
      # response_format="text",
      temperature=0.0,
      # stream=True,
    )
    response_content = response_content.choices[0].message.content
    # print(response_content)
    return response_content
  except Exception as e:
      print(f"An error occurred: {e}")
   

In [5]:
import sys
module_path = '../../package'
if module_path not in sys.path:
    sys.path.append(module_path)
from jupyter_ui import open_dir


open_dir.open_dir_ui()
# # "JobDetail","List","HunterCenter"

GridBox(children=(Button(button_style='success', description='选择解析目录', style=ButtonStyle(), tooltip='选择目录，目录下所…

In [6]:
from pathlib import Path


def get_start_dir():
    text = open_dir.dir_text.value
    start_dir = Path(text)
    return start_dir


def get_exclude_dirs():
    return ["JobDetail", "List", "HunterCenter"]


def write_res(content, write_path):
    write_path.write_text(content)


def run(dir_path, is_write=True, include_dirs=[], exclude_dirs=[]):
    # 判断是否是目录
    if not dir_path.is_dir():
        content = dir_path.read_text()
        res_content = prompt_model(content)

        write_path = dir_path if is_write else Path("../data/res.tsx")
        write_res(res_content, write_path)
        exit(1)

    # # 遍历目录
    for file_path in dir_path.rglob("*"):
        # 检查当前路径是否在需要排除的目录中
        if any(excluded in file_path.parts for excluded in exclude_dirs):
            continue
        # 检查文件后缀名
        if file_path.suffix in include_dirs:
            # 打印文件路径
            print(f"Found file: {file_path}")
            # 读取文件内容
            try:
                content = file_path.read_text()
                res_content = prompt_model(content)
                # 生成结果写入文件中
                write_path = file_path if is_write else Path("../data/res.tsx")
                write_res(res_content, write_path)
                print("------")  # 分隔线，用于区分不同文件的内容
            except Exception as e:
                print(f"Error reading file {file_path}: {e}")


dir_path = Path("../data/1.tsx") # get_start_dir()
run(dir_path, is_write=False, include_dirs=[".ts"], exclude_dirs=[])